In [1]:
import eugene
import torch
import json
import lightning.pytorch as pl
import xarray as xr
import numpy as np

c:\Users\Lab\anaconda3\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Load data into SeqData xarray

In [2]:
import os, wget
if not os.path.exists("test-sample.pt"):
    wget.download("https://github.com/lucidrains/enformer-pytorch/raw/main/data/test-sample.pt")

In [3]:
data = torch.load("test-sample.pt")

In [4]:
seq = data["sequence"].cpu().numpy()[np.newaxis, :, :]
target = data["target"].cpu().numpy()
sdata = xr.Dataset(
    data_vars=dict(
        ohe_seq=(["_sequence", "length", "_ohe"], seq),
        target=(["target_length", "_targets"], target),
        train_val=(["_sequence"], [True])
    ),
    attrs=dict(max_jitter=0),
)

In [5]:
sdata

<xarray.Dataset>
Dimensions:    (_sequence: 1, length: 131072, _ohe: 4, target_length: 896,
                _targets: 5313)
Dimensions without coordinates: _sequence, length, _ohe, target_length, _targets
Data variables:
    ohe_seq    (_sequence, length, _ohe) float32 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    target     (target_length, _targets) float32 0.09924 0.09271 ... 0.0 0.0
    train_val  (_sequence) bool True
Attributes:
    max_jitter:  0

# Set up model using Basenji2

In [6]:
# Change basenji2-pytorch-main to basenji2 and add __init__.py
from basenji2.basenji2_pytorch import Basenji2, params

In [7]:
model_weights = "basenji2.pth"

with open(params) as params_open:
    model_params = json.load(params_open)["model"]

basenji2 = Basenji2(model_params)
basenji2.load_state_dict(torch.load(model_weights), strict=False)
basenji2 = basenji2.cpu()

In [8]:
class Basenji2Wrap(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = basenji2
        self.input_len = sdata["ohe_seq"].values.shape[1]
        self.output_dim = [1024]

    def forward(self, x):
        print(x.shape)
        x = np.transpose(np.squeeze(x))
        print(x.shape)
        return self.model(x)

In [9]:
from eugene.models import SequenceModule
model = SequenceModule(
    arch = Basenji2Wrap()
)

In [10]:
model.summary()

Model: Basenji2Wrap
Sequence length: 131072
Output dimension: [1024]
Task: regression
Loss function: mse_loss
Optimizer: Adam
	Optimizer parameters: {}
	Optimizer starting learning rate: 0.001
Scheduler: None
	Scheduler parameters: {}
Metric: r2score
	Metric parameters: {}
Seed: None
Parameters summary:


  | Name         | Type         | Params
----------------------------------------------
0 | arch         | Basenji2Wrap | 30.1 M
1 | train_metric | R2Score      | 0     
2 | val_metric   | R2Score      | 0     
3 | test_metric  | R2Score      | 0     
----------------------------------------------
30.1 M    Trainable params
0         Non-trainable params
30.1 M    Total params
120.353   Total estimated model params size (MB)

# Test model

In [11]:
out = model.predict(x = sdata["ohe_seq"].values)

Predicting on batches: 0it [00:00, ?it/s]

torch.Size([1, 131072, 4])
torch.Size([4, 131072])


RuntimeError: running_mean should contain 131072 elements not 288